In [1]:
import pandas as pd
import numpy as np
import requests as rq
from shapely.geometry import Point
import geopandas as gpd
import re
import os
import geopandas as gpd

In [2]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

In [3]:
def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)
def distance_meters(mercator_start,mercator_finish):
    return mercator_start.distance(mercator_finish)

In [4]:
def sports_centres():
    instalaciones_dep_data=rq.get('https://datos.madrid.es/egob/catalogo/200215-0-instalaciones-deportivas.json')
    instalaciones_dep_data=instalaciones_dep_data.json()
    instalaciones_dep_data = pd.json_normalize(instalaciones_dep_data['@graph'])
    instalaciones_dep_data=instalaciones_dep_data.drop(['@id', '@type','id', 'relation', 'address.district.@id', 'address.area.@id', 'address.locality', 'address.postal-code', 'organization.organization-desc', 'organization.accesibility', 'organization.schedule', 'organization.services','organization.organization-name'], axis='columns')
    instalaciones_dep_data['address.street-address']=instalaciones_dep_data['address.street-address'].str.title()
    instalaciones_dep_data.dropna(subset=['location.latitude','location.longitude'], inplace=True)
    instalaciones_dep_data1=instalaciones_dep_data.rename(columns={'location.latitude':'lat_start','location.longitude':'long_start'})
    instalaciones_dep_data1["Punto Inicio"] = instalaciones_dep_data1.apply(lambda x: to_mercator(x["lat_start"],x["long_start"]), axis = 1)
    return instalaciones_dep_data1

In [4]:
sports_centres()

,title,address.street-address,lat_start,long_start,Punto Inicio
0,Instalación Deportiva Básica Embalse de San Juan,Calle Embalse De San Juan 1,40.362643,-3.598148,POINT (4493148.849867125 -400807.5874588013)
1,Instalación Deportiva Básica Alto del Retiro,Calle Alto Del Retiro 13 C/V Calle Antonio Gades,40.367515,-3.588265,POINT (4493691.180400329 -399705.1621850753)
2,Instalación Deportiva Básica Avenida de Cerro ...,Avenida Cerro Milano 1,40.368619,-3.618357,POINT (4493814.110132848 -403061.6411942258)
3,Instalación Deportiva Básica Avenida de las Su...,Avenida Avenida De Las Suertes,40.370297,-3.612067,POINT (4494000.934882545 -402360.1094749893)
4,Instalación Deportiva Básica Avenida del Ensan...,Avenida Ensanche De Vallecas 11,40.370297,-3.612067,POINT (4494000.934882545 -402360.1094749893)
...,...,...,...,...,...
507,Instalación Deportiva Municipal Pádel Marqués...,Calle Ribera Del Manzanares 1,40.422170,-3.724405,POINT (4499775.434394474 -414891.1477215095)
508,Instalación Deportiva Municipal Rocódromo 'Roc...,Avenida Manzanares 108,40.399273,-3.718824,POINT (4497226.462121102 -414268.572572413)
510,Parque Lineal UZI 0.06 Arroyofresno,Calle Maria De Maeztu 1,40.487147,-3.732863,POINT (4507008.539681911 -415834.6524269146)
511,Pista de Patinaje Paco de Lucía,Auditorio Paco De Lucia,40.385278,-3.764155,POINT (4495668.617279258 -419325.5917997768)


In [9]:
def bicimad_stations():
    bicimad= pd.read_json("../ih_datamadpt1121_project_m1/Estaciones.json")
    bicimad=bicimad.drop(['light', 'number', 'activate', 'no_available', 'total_bases', 'dock_bikes', 'free_bases','reservations_count','geometry_type'], axis='columns')
    bicimad['long_finish']=bicimad['geometry_coordinates'].map(lambda x:x.split(',')[0].replace('[','')).astype(float)
    bicimad['lat_finish']=bicimad['geometry_coordinates'].map(lambda x:x.split(',')[1].replace(']','')).astype(float)
    bicimad=bicimad.drop(['id','geometry_coordinates'],axis='columns')
    bicimad["Punto Final"] = bicimad.apply(lambda x: to_mercator(x["lat_finish"],x["long_finish"]), axis = 1)
    return bicimad

In [10]:
bicimad_stations()

,name,address,long_finish,lat_finish,Punto Final
0,Puerta del Sol A,Puerta del Sol nº 1,-3.701834,40.417214,POINT (4499223.648366921 -412373.2851572837)
1,Puerta del Sol B,Puerta del Sol nº 1,-3.701603,40.417313,POINT (4499234.670122574 -412347.498528782)
2,Miguel Moya,Calle Miguel Moya nº 1,-3.705842,40.420589,POINT (4499599.340516399 -412820.3206106317)
3,Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,-3.706917,40.430294,POINT (4500679.707306497 -412940.3068148255)
4,Malasaña,Calle Manuela Malasaña nº 5,-3.702587,40.428552,POINT (4500485.866677178 -412457.3286497012)
...,...,...,...,...,...
259,Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,-3.726990,40.443750,POINT (4502177.655770458 -415179.5220676512)
260,Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,-3.726930,40.443420,POINT (4502140.920338497 -415172.8287427764)
261,Facultad Biología,Calle José Antonio Novais frente al nº 12,-3.727295,40.448332,POINT (4502687.743941171 -415213.4906984274)
262,Facultad Derecho,Avenida Complutense nº 23,-3.729370,40.451090,POINT (4502994.74083288 -415445.0243226733)


In [17]:
def clean_data():
    df_final = pd.merge(sports_centres(), bicimad_stations(), how='cross')
    df_final['Distancia']=df_final.apply(lambda x: distance_meters(x['Punto Inicio'], x['Punto Final']), axis=1)
    df_final1= df_final.drop(['lat_start','long_start','Punto Inicio', 'Punto Final', 'long_finish', 'lat_finish' ], axis='columns')
    df_final1=df_final1.rename(columns={'title':'Place of interest', 'address.street-address':'Place address','name':'BiciMAD station','address':'Station location'})
    df_final1.insert(1, 'Type of place', 'Instalación deportiva')
    return df_final1

In [18]:
clean_data()

,Place of interest,Type of place,Place address,BiciMAD station,Station location,Distancia
0,Instalación Deportiva Básica Embalse de San Juan,Instalación deportiva,Calle Embalse De San Juan 1,Puerta del Sol A,Puerta del Sol nº 1,13064.016996
1,Instalación Deportiva Básica Embalse de San Juan,Instalación deportiva,Calle Embalse De San Juan 1,Puerta del Sol B,Puerta del Sol nº 1,13046.331120
2,Instalación Deportiva Básica Embalse de San Juan,Instalación deportiva,Calle Embalse De San Juan 1,Miguel Moya,Calle Miguel Moya nº 1,13635.049959
3,Instalación Deportiva Básica Embalse de San Juan,Instalación deportiva,Calle Embalse De San Juan 1,Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,14279.940222
4,Instalación Deportiva Básica Embalse de San Juan,Instalación deportiva,Calle Embalse De San Juan 1,Malasaña,Calle Manuela Malasaña nº 5,13767.653594
...,...,...,...,...,...,...
135163,Pista de petanca Centro de Mayores La Vaguada,Instalación deportiva,Avenida Monforte De Lemos 36,Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,4448.637413
135164,Pista de petanca Centro de Mayores La Vaguada,Instalación deportiva,Avenida Monforte De Lemos 36,Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,4477.987873
135165,Pista de petanca Centro de Mayores La Vaguada,Instalación deportiva,Avenida Monforte De Lemos 36,Facultad Biología,Calle José Antonio Novais frente al nº 12,4023.270682
135166,Pista de petanca Centro de Mayores La Vaguada,Instalación deportiva,Avenida Monforte De Lemos 36,Facultad Derecho,Avenida Complutense nº 23,3901.183322


In [27]:
def distancia_minima():
    x=str(input('Introduce una Instalación Deportiva: '))
    y = clean_data()[clean_data()['Place of interest']==x]
    return y.sort_values(by = "Distancia", ascending = True).groupby('Place of interest')['Place of interest','Type of place','Place address', 'Distancia','BiciMAD station', 'Station location'].nth(0).drop(["Distancia"], axis = "columns")

In [28]:
distancia_minima()

Introduce una Instalación Deportiva: Pista de petanca Centro de Mayores La Vaguada


C:\Users\Diego Merello\miniconda3\envs\project_1\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.


,Place of interest,Type of place,Place address,BiciMAD station,Station location
Place of interest,,,,,
Pista de petanca Centro de Mayores La Vaguada,Pista de petanca Centro de Mayores La Vaguada,Instalación deportiva,Avenida Monforte De Lemos 36,Remonta,Calle Azucenas nº 1


In [35]:
def bicimad_todas():
    return (clean_data().sort_values(by = "Distancia", ascending = True).groupby('Place of interest')['Type of place','Place address', 'Distancia','BiciMAD station', 'Station location'].nth(0).drop(["Distancia"], axis = "columns"))


In [36]:
%%time
bicimad_todas()

C:\Users\Diego Merello\miniconda3\envs\project_1\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


Wall time: 2min 25s


,Type of place,Place address,BiciMAD station,Station location
Place of interest,,,,
Instalación Deportiva Básica Embalse de San Juan,Instalación deportiva,Calle Embalse De San Juan 1,Pavones,Calle Hacienda de Pavones nº 342
Instalación Deportiva Básica Alto del Retiro,Instalación deportiva,Calle Alto Del Retiro 13 C/V Calle Antonio Gades,Pavones,Calle Hacienda de Pavones nº 342
Instalación Deportiva Básica Avenida de Cerro Milano,Instalación deportiva,Avenida Cerro Milano 1,Pavones,Calle Hacienda de Pavones nº 342
Instalación Deportiva Básica Avenida de las Suertes,Instalación deportiva,Avenida Avenida De Las Suertes,Pavones,Calle Hacienda de Pavones nº 342
Instalación Deportiva Básica Avenida del Ensanche de Vallecas,Instalación deportiva,Avenida Ensanche De Vallecas 11,Pavones,Calle Hacienda de Pavones nº 342
...,...,...,...,...
Instalación Deportiva Municipal Pádel Marqués de Monistrol,Instalación deportiva,Calle Ribera Del Manzanares 1,Paseo de la Florida,Paseo de la Florida nº 8
Instalación Deportiva Municipal Rocódromo 'Roc 30',Instalación deportiva,Avenida Manzanares 108,Marqués de Vadillo,Glorieta Marqués de Vadillo nº 9
Parque Lineal UZI 0.06 Arroyofresno,Instalación deportiva,Calle Maria De Maeztu 1,Facultad Derecho,Avenida Complutense nº 23
